<a href="https://colab.research.google.com/github/JonatasLemos/DataAnalysisProjects/blob/main/wikipediaAnalysis/CitizensWikipediaPart1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning - Scraped data from wikipedia using Scrapy crawler


In [ ]:
import pandas as pd
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Crawler details

```
import scrapy

class WikipediaSpider(scrapy.Spider):
    name = 'wikipedia'
    start_urls = ['https://pt.wikipedia.org/w/index.php?title=Categoria:Naturais_de_Minas_Gerais']
    def parse(self, response):

        yield {'nome': response.css('div.CategoryTreeItem > a::text').getall(),
              'numero':response.css('div.CategoryTreeItem > span[dir~=ltr]::text').getall() }

        extra_urls = ["Itamonte%0ANaturais+de+Itamonte", "Sao+Roque+Minas%0ANaturais+de+São+Roque+de+Minas"]
        for url in extra_urls:
            yield scrapy.Request(
                url=f"{self.start_urls[0]}&subcatfrom={url}#mw-subcategories",
                callback=self.parse)
```



## Creating DF from JSON file with scrapped data


In [ ]:
path_start = "/content/drive/MyDrive/Colab Data/"
f = open(f'{path_start}wiki.json')
cities = json.load(f)

In [ ]:
len(cities[0]["nome"])

200

In [ ]:
city_names,n_of_citizens = [],[]
for i in range(len(cities)):
  city_names.extend(cities[i]["nome"])
  n_of_citizens.extend(cities[i]["numero"])

In [ ]:
df_cities = pd.DataFrame({"CIDADE":city_names,"CIDADAOS ILUSTRES":n_of_citizens})
df_cities.head()

,CIDADE,CIDADAOS ILUSTRES
0,Naturais de Minas Gerais por ocupação,(43 C)
1,Naturais de Abadia dos Dourados,(4 P)
2,Naturais de Abaeté,(11 P)
3,Naturais de Abre-Campo,(2 P)
4,Naturais de Acaiaca,(2 P)


In [ ]:
df_cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   CIDADE             451 non-null    object
 1   CIDADAOS ILUSTRES  451 non-null    object
dtypes: object(2)
memory usage: 7.2+ KB


## Eliminating unwanted chars and unwanted records

In [ ]:
df_cities["CIDADE"].replace(to_replace='\(.*|Naturais de', value='',regex=True, inplace=True)
df_cities["CIDADE"] = df_cities["CIDADE"].str.strip()
df_cities.head(3)

,CIDADE,CIDADAOS ILUSTRES
0,Minas Gerais por ocupação,(43 C)
1,Abadia dos Dourados,(4 P)
2,Abaeté,(11 P)


In [ ]:
df_cities.drop([0,242],inplace=True)
df_cities["CIDADAOS ILUSTRES"].unique()
unwantedIndexes = df_cities[df_cities['CIDADAOS ILUSTRES'] == "(vazia)"].index
df_cities.drop(unwantedIndexes , inplace=True) 
df_cities.reset_index(inplace=True,drop=True)

## Reading population CSV and converting it to datraframe
### Filtering and applying some changes

In [ ]:
cities_pop = pd.read_csv(f"{path_start}sorted_cities_pop.csv")
cities_pop = cities_pop[cities_pop["STATE"] == "MG"]
cities_pop = cities_pop.drop(["Unnamed: 0","DISTANCE POP","STATE"],axis=1)
cities_pop.head()

,NAME,POPULAÇÃO ESTIMADA
2114,Belo Horizonte/MG,2502557
2115,Contagem/MG,648766
2116,Betim/MG,417307
2117,Ribeirão das Neves/MG,322659
2118,Santa Luzia/MG,216254


In [ ]:
cities_pop["NAME"] = cities_pop["NAME"].str.split("/").apply(lambda x:x[0])
cities_pop.reset_index(inplace=True,drop=True)
cities_pop.head(3)

,NAME,POPULAÇÃO ESTIMADA
0,Belo Horizonte,2502557
1,Contagem,648766
2,Betim,417307


## Correcting wrong names

In [ ]:
wrong_names = df_cities[~df_cities['CIDADE'].isin(cities_pop['NAME'])]["CIDADE"].to_list()
wrong_names

['Abre-Campo',
 'Amparo da Serra',
 'Bocaiuva',
 'Brazópolis',
 'Galileia',
 'São Tomé das Letras',
 'Passa-Quatro',
 'São João del-Rei']

In [ ]:
correct_names = ["Abre Campo", "Amparo do Serra", "Bocaiúva","Brasópolis","Galiléia",
                "São Thomé das Letras","Passa Quatro", "São João del Rei"]       
df_cities['CIDADE'].replace(wrong_names,correct_names,inplace=True)
df_cities["CIDADAOS ILUSTRES"].replace(to_replace='\(|P\)', value='', regex=True,inplace=True)
df_cities.head()

,CIDADE,CIDADAOS ILUSTRES
0,Abadia dos Dourados,4
1,Abaeté,11
2,Abre Campo,2
3,Acaiaca,2
4,Açucena,1


In [ ]:
cities_pop.rename(columns={"NAME": "CIDADE"},inplace=True)

In [ ]:
df_cities["CIDADAOS ILUSTRES"].unique()

array(['4 ', '11 ', '2 ', '1 ', '12 ', '6 ', '10 ', '18 ', '3 ', '8 ',
       '31 ', '24 ', '48 ', '624 ', '7 ', '5 ', '23 ', '19 ', '26 ',
       '15 ', '21 ', '38 ', '20 ', '17 ', '35 ', '33 ', '9 ', '13 ',
       '28 ', '72 ', '56 ', '154 ', '37 ', '22 ', '34 ', '29 ', '16 ',
       '58 ', '67 '], dtype=object)

In [ ]:
df_cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 447 entries, 0 to 446
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   CIDADE             447 non-null    object
 1   CIDADAOS ILUSTRES  447 non-null    object
dtypes: object(2)
memory usage: 7.1+ KB


## Merging DFs using outer join 

In [ ]:
df_cities = df_cities.merge(cities_pop,how='outer')
df_cities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 853 entries, 0 to 852
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   CIDADE              853 non-null    object
 1   CIDADAOS ILUSTRES   447 non-null    object
 2   POPULAÇÃO ESTIMADA  853 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 26.7+ KB


In [ ]:
df_cities['CIDADAOS ILUSTRES'].fillna(0,inplace=True)

## Exporting final DF to CSV

In [ ]:
df_cities["CIDADAOS ILUSTRES"] = df_cities["CIDADAOS ILUSTRES"].astype(int)
df_cities.sort_values(by="CIDADAOS ILUSTRES",ascending=False,inplace=True)
df_cities.to_csv(f'{path_start}merged_cities_famous.csv')